# IMPORTS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Fetching the filename, Number of Clusters, Max Iterations

In [ ]:
filePath = input("Please enter file name: ") #"cho.txt"
num_clusters = int(input("Please enter the number of cluster you would like: "))
max_iter = int(input("Please enter the max number of iterations: "))

# K-Means Algorithm, Jaccard Co-eff & Rand Index Value Calculation
   (Need to provide input for centroids - Scroll Down Please)

In [ ]:
file = open(filePath, "r")
lines = file.readlines()

df1 = ""
# splittng the line into individual data
flag1 = 2
for line in lines:
    if(" " in line):
        flag1 = 0
    elif("\t" in line):
        flag1 = 1
    break

if(flag1 == 0):
    for line in lines:
        df1 = pd.read_csv(filePath, sep = ' ', lineterminator = '\n', header = None)
elif(flag1 == 1):
    for line in lines:
        df1 = pd.read_csv(filePath, sep = '\t', lineterminator = '\n', header = None)
else:
    print("Data should be sperated by \t or single space!")
    
df2 = df1.copy()

centroids = {}
#centroids[0] = 
clusters = {}
old_centroids = {}
(rowcount,colcount) = (df2.shape)
print("Do you want to use default initial centroids (1) or enter your own centroids (2)?")
cent_type = int(input("1 for system calculated or 2 for user centroids: "))
for i in range(0,num_clusters):
    clusters[i] = [] #[df2.iloc[temp1,0]]

if(cent_type==1):
    temp1 = 0; step1 = int(rowcount/num_clusters);
    for i in range(0,num_clusters):
        centroids[i] = df2.iloc[temp1,2:].values
        #temp_sse[i] = []
        temp1 += step1
elif(cent_type==2):
    str1 = input("Please enter data point (gene) id in a c1,c2,c3 format:")
    str2 = str1.split(",")
    for i in range(len(str2)):
        pt1 = int(str2[i])-1
        centroids[i] = df2.iloc[pt1,2:].values
    
dict1 = {}
dict2 = {}
for j in range(0,rowcount):
    dict1[df2.iloc[j,0]] = df2.iloc[j,2:].values
    dict2[df2.iloc[j,0]] = df2.iloc[j,1]
    
flag = 1
iter1 = 0
while(flag==1):
    #print("Iteration Start")
    iter1 +=1
    #sse["iter"+str(i+1)] = {}
    
    old_centroids = centroids.copy()
    #sse["iter"+str(i+1)] = {}
    for i2 in range(0,num_clusters):
        clusters[i2] = []

    for i3 in dict1.keys():
        dist_temp = 10000
        centr_temp = 0
        pt1 = dict1[i3]
        for j in centroids.keys():
            pt2 = centroids[j]
            c = pt1-pt2
            sum1 = 0
            for k in c:
                sum1 += (k**2)
                #print(sum**(1/2))
                #print(c)
            sum1 = sum1**(1/2)
            if(sum1<dist_temp):
                dist_temp = sum1
                centr_temp = j
        clusters[centr_temp].append(i3)
        #temp_sse[centr_temp] = (dist_temp)
    #sse["iter"+str(i+1)] = temp_sse
    for l in clusters.keys():
        list_temp = clusters[l]
        s3 = dict1[list_temp[0]]
        for m in range(1,len(list_temp)):
            s3 = s3 + dict1[list_temp[m]]
        centroids[l] = (s3/len(list_temp))
        
    list1 = []
    for cnt in range(0,len(centroids)):
        list1.append(np.array_equal(old_centroids[cnt],centroids[cnt]))
    if(False in list1):
        flag = 1
    else:
        flag = 0
    if(iter1==max_iter):
        flag = 0
    else:
        flag = 1
print()
print("----------------------------------------------------------------------------------------------------")
print("Final Centroids -> "+str(centroids))
print("Final Clusters -> "+str(clusters))
print("----------------------------------------------------------------------------------------------------")

ground_truth = (list(df2.iloc[:,1].values))
observed_truth = []
for p1 in range(df2.shape[0]):
    observed_truth.append(-1)
    
for i in clusters.keys():
    list1 = clusters[i]
    for j in list1:
        observed_truth[j-1] = (i+1)
#print(observed_truth)

def cjr(ground_truth,observed_truth):
    true_pos = 0
    true_neg = 0
    false_pos=0
    false_neg=0
    for i in range(df2.shape[0]):
        for j in range(df2.shape[0]):
            if ground_truth[i]==ground_truth[j]:
                if observed_truth[i]==observed_truth[j]:
                    true_pos=true_pos+1
                else:
                    false_neg=false_neg+1
            elif ground_truth[i]!=ground_truth[j]:
                if observed_truth[i]==observed_truth[j]:
                    false_pos=false_pos+1
                else:
                    true_neg=true_neg+1
    jaccard_coeff = (true_pos)/(true_pos+false_pos+false_neg)
    rand_index = (true_pos+true_neg)/(true_pos+true_neg+false_pos+false_neg)
    return jaccard_coeff,rand_index

jaccard_coeff,rand_index = cjr(ground_truth,observed_truth)

# Data Visualization and Jaccard Co-eff & Rand Index Value

In [ ]:
from sklearn.decomposition import PCA
observed_truth_nparr = np.asarray(observed_truth)
df3 = df2.copy()
X_df = df3.iloc[:,2:].values
ldf = pd.DataFrame(observed_truth_nparr,columns=["Label"])


X_df = df2.iloc[:,2:].values
#X_df = StandardScaler().fit_transform(X_df)
if(df2.shape[1]>4):
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(X_df)
    principalDf = pd.DataFrame(data = principalComponents,columns = ['PC1', 'PC2'])
    finalDf = pd.concat([principalDf, ldf["Label"]], axis = 1)
    print()
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    til1 = "K-Means ("+str(filePath)+") with "+str(num_clusters)+" clusters"
    ax.set_title(til1, fontsize = 20)
    targets = list(set(observed_truth_nparr))
    for pt in targets:
        indicesToKeep = finalDf['Label'] == pt
        ax.scatter(finalDf.loc[indicesToKeep, 'PC1'], finalDf.loc[indicesToKeep, 'PC2'], s = 50)
    ax.legend(targets)
    ax.grid()

elif(df2.shape[1]==4):
    dfnew = df2.iloc[:,2:4]
    dfnew.columns = ['X','Y']
    finalDf = pd.concat([dfnew, ldf["Label"]], axis = 1)
    print()
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('X', fontsize = 15)
    ax.set_ylabel('Y', fontsize = 15)
    til1 = "K-Means ("+str(filePath)+") with "+str(num_clusters)+" clusters"
    ax.set_title(til1, fontsize = 20)
    targets = list(set(observed_truth_nparr))
    for pt in targets:
        indicesToKeep = finalDf['Label'] == pt
        ax.scatter(finalDf.loc[indicesToKeep, 'X'], finalDf.loc[indicesToKeep, 'Y'], s = 50)
    ax.legend(targets)
    ax.grid()
    
print("Jaccard Coefficient = ",jaccard_coeff)
print("Rand Index = ",rand_index)